In [122]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast
0.19.0


In [123]:
import numpy as np
import pandas as pd
import joblib
import np.np.np.np.nannyml as nml
from sklearn.ensemble import RandomForestRegressor
from skforecast.datasets import fetch_dataset
from skforecast.drift_detection import PopulationDriftDetector

ModuleNotFoundError: No module named 'np'

In [ ]:
data = joblib.load(r'/home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/tests/tests_population_drift/fixture_data_population_drift.joblib')
data['weather'] = data['weather'].astype('category')
data_multiseries = pd.concat(
    [
        data.assign(series='series_1'),
        data.assign(series='series_2'),
        data.assign(series='series_3')
    ]
).set_index('series', append=True).swaplevel(0, 1)

results_nannyml = joblib.load(r'/home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/tests/tests_population_drift/fixture_results_nannyml.joblib')

In [ ]:
detector = nml.UnivariateDriftCalculator(
    column_names=data_train.columns.tolist(),
    timestamp_column_name="date_time",
    chunk_period='M',
    categorical_methods=['chi2', 'jensen_shannon'],
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
)
detector.fit(reference_data=data_train.reset_index())
results_nannyml = detector.calculate(data=data_new.reset_index())
results_nannyml = results_nannyml.filter(period='analysis').to_df(multilevel=False)

In [ ]:
reference = results_nannyml.filter(period='reference').to_df(multilevel=False).filter(regex="date")
reference['chunk_end_date'].dt.round('1s').

0    2011-02-01
1    2011-03-01
2    2011-04-01
3    2011-05-01
4    2011-06-01
5    2011-07-01
6    2011-08-01
7    2011-09-01
8    2011-10-01
9    2011-11-01
10   2011-12-01
11   2012-01-01
12   2012-02-01
Name: chunk_end_date, dtype: datetime64[ns]

In [ ]:
results_skforecast.filter(regex="chunk")

,chunk,chunk_start,chunk_end
0,0,2012-01-01 12:00:00,2012-01-31 23:00:00
1,1,2012-02-01 00:00:00,2012-02-29 23:00:00
2,2,2012-03-01 00:00:00,2012-03-31 23:00:00
3,3,2012-04-01 00:00:00,2012-04-30 23:00:00
4,4,2012-05-01 00:00:00,2012-05-31 23:00:00
...,...,...,...
127,7,2012-08-01 00:00:00,2012-08-31 23:00:00
128,8,2012-09-01 00:00:00,2012-09-30 23:00:00
129,9,2012-10-01 00:00:00,2012-10-31 23:00:00
130,10,2012-11-01 00:00:00,2012-11-30 23:00:00


In [124]:
data_train = data.iloc[: len(data) // 2].copy()
data_new  = data.iloc[len(data) // 2 :].copy()
data_train['weather'] = data_train['weather'].astype('category')
data_new['weather'] = pd.Categorical(
    data_new['weather'], categories=data_train['weather'].cat.categories
)

detector = PopulationDriftDetector(
    chunk_size='MS',            
    threshold=3,
    threshold_method='std'
)
detector.fit(data_train)
results_skforecast, _ = detector.predict(data_new)

features = results_skforecast["feature"].unique()
for feature in features:
    print(f"Feature: {feature}")
    cols = [
        "chunk_index",
        "chunk_start_date",
        "chunk_end_date",
        f"{feature}_kolmogorov_smirnov_value",
        f"{feature}_kolmogorov_smirnov_upper_threshold",
        f"{feature}_jensen_shannon_value",
        f"{feature}_jensen_shannon_upper_threshold",
        f"{feature}_chi2_value",
        f"{feature}_chi2_upper_threshold",
    ]
    # select columns if they exist in results_skforecast
    cols = [col for col in cols if col in results_nannyml.columns]
    df_nannyml = results_nannyml[cols].copy()
    df_nannyml = df_nannyml.rename(
        columns={
            "chunk_index": "chunk",
            "chunk_start_date": "chunk_start",
            "chunk_end_date": "chunk_end",
            f"{feature}_kolmogorov_smirnov_value": "ks_statistic",
            f"{feature}_kolmogorov_smirnov_upper_threshold": "ks_threshold",
            f"{feature}_jensen_shannon_value": "js_statistic",
            f"{feature}_jensen_shannon_upper_threshold": "js_threshold",
            f"{feature}_chi2_value": "chi2_statistic",
            f"{feature}_chi2_upper_threshold": "chi2_threshold",
        }
    )
    df_nannyml["feature"] = feature

    if "chi2_statistic" not in df_nannyml.columns:
        df_nannyml["chi2_statistic"] = np.nan
        df_nannyml["chi2_threshold"] = np.nan
        df_nannyml = df_nannyml.astype(
            {"chi2_statistic": float, "chi2_threshold": float}
        )
    if "ks_statistic" not in df_nannyml.columns:
        df_nannyml["ks_statistic"] = np.nan
        df_nannyml["ks_threshold"] = np.nan
        df_nannyml = df_nannyml.astype(
            {"ks_statistic": float, "ks_threshold": float}
        )
    if "js_statistic" not in df_nannyml.columns:
        df_nannyml["js_statistic"] = np.nan
        df_nannyml["js_threshold"] = np.nan
        df_nannyml = df_nannyml.astype(
            {"js_statistic": float, "js_threshold": float}
        )

    df_nannyml = df_nannyml.astype(
        {"chi2_statistic": float, "chi2_threshold": float,
            "ks_statistic": float, "ks_threshold": float,
            "js_statistic": float, "js_threshold": float}
    )

    df_nannyml = df_nannyml[
        [
            "chunk",
            # "chunk_start",
            # "chunk_end",
            "feature",
            "ks_statistic",
            #"ks_threshold",
            "chi2_statistic",
            # "chi2_threshold",
            "js_statistic",
            "js_threshold",
        ]
    ]

    df_skforecast = results_skforecast.query(f"feature == '{feature}'")[
        [
            "chunk",
            # "chunk_start",
            # "chunk_end",
            "feature",
            "ks_statistic",
            #"ks_threshold",
            "chi2_statistic",
            # "chi2_threshold",
            "js_statistic",
            "js_threshold",
        ]
    ].reset_index(drop=True)

    display(df_nannyml.head(3))
    display(df_skforecast.head(3))
    pd.testing.assert_frame_equal(
        df_nannyml, 
        df_skforecast
    )

Feature: holiday


Feature: holiday


AttributeError: 'Result' object has no attribute 'columns'

In [88]:
df_nannyml

,chunk,feature,ks_statistic,ks_threshold,chi2_statistic,js_statistic,js_threshold
0,0,month,0.913817,1.0,NaN,0.884780,0.895332
1,1,month,0.837209,1.0,NaN,0.894822,0.895332
2,2,month,0.752394,1.0,NaN,0.886196,0.895332
3,3,month,0.670315,1.0,NaN,0.889046,0.895332
4,4,month,0.585499,1.0,NaN,0.886196,0.895332
5,5,month,0.503420,1.0,NaN,0.889046,0.895332
6,6,month,0.496580,1.0,NaN,0.886196,0.895332
7,7,month,0.581395,1.0,NaN,0.886196,0.895332
8,8,month,0.666211,1.0,NaN,0.889046,0.895332
9,9,month,0.748290,1.0,NaN,0.886196,0.895332


In [116]:
dt = pd.Timestamp('2011-01-31 23:59:59.999999999')
print(f"Original: {dt}")
dt.round(pd.Timedelta('1s'))

Original: 2011-01-31 23:59:59.999999999


Timestamp('2011-02-01 00:00:00')